**Chapter 15 – Processing Sequences Using RNNs and CNNs**

# Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
# Common imports
import numpy as np

np.random.seed(42)
tf.random.set_seed(42)

import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Basic RNNs

### Generate the Dataset

In [ ]:
windows = 50
horizon = 1 

In [ ]:
def generate_time_series(batch_size, windows):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, windows)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 9 + 9))  #   wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 19 + 19)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, windows) - 0.5)   # + noise
    return series[..., np.newaxis].astype(np.float32)

In [ ]:
np.random.seed(42)



series = generate_time_series(3000, windows + horizon)
X_train, y_train = series[:1000, :windows], series[:1000, -horizon:, 0]
X_valid, y_valid = series[1000:2000, :windows], series[1000:2000, -horizon:, 0]
X_test, y_test = series[2000:, :windows], series[2000:, -horizon:, 0]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
def plot_series(series, windows, y=None, y_pred=None, x_label="$t$", y_label="$x(t)$"):
    plt.plot(series, ".-")
    if y is not None:
        plt.plot(windows, y, "bx", markersize=10)
    if y_pred is not None:
        plt.plot(windows, y_pred, "ro")
    plt.grid(True)
    if x_label:
        plt.xlabel(x_label, fontsize=16)
    if y_label:
        plt.ylabel(y_label, fontsize=16, rotation=0)
    plt.hlines(0, 0, 100, linewidth=1)
    plt.axis([0, windows + 1, -1, 1])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(12, 4))
for col in range(3):
    plt.sca(axes[col])
    #plot_series(X_valid[col, :, 0], y_valid[col, 0], y_label=("$x(t)$" if col==0 else None))
    plot_series(X_valid[col, :, 0], windows, y_label=("$x(t)$" if col==0 else None))

plt.show()

In [ ]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

### Computing Some Baselines

Naive predictions (just predict the last observed value):

In [ ]:
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

In [ ]:
plot_series(X_valid[0, :, 0],windows,y_valid[0, 0], y_pred[0, 0])
plt.show()

Linear predictions:

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model_1 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[windows, 1]),
    keras.layers.Dense(horizon)
])

In [ ]:
mae = tf.keras.metrics.MeanAbsoluteError()

In [ ]:
#model_1.compile(loss="mse", optimizer=keras.optimizers.Adam(lr=0.01), metrics=[last_time_step_mse])

model_1.compile(loss="mse", optimizer=keras.optimizers.Adam(lr=0.01), metrics=[mae])

history_1 = model_1.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

In [ ]:
model_1.summary()

In [ ]:
tf.keras.utils.plot_model(model_1,  show_shapes=True)

In [ ]:
model_1.evaluate(X_valid, y_valid)

In [ ]:
def plot_learning_curves(loss, val_loss):
    plt.plot(np.arange(len(loss)) + 0.5, loss, "b.-", label="Training loss")
    plt.plot(np.arange(len(val_loss)) + 1, val_loss, "r.-", label="Validation loss")
    plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    plt.gca().yaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    plt.axis([1, 30, 0, 0.05])
    plt.legend(fontsize=14)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)

In [ ]:
plot_learning_curves(history_1.history["loss"], history_1.history["val_loss"])
plt.show()

In [ ]:
y_pred = model_1.predict(X_valid)
plot_series(X_valid[0, :, 0],windows,y_valid[0, 0], y_pred[0, 0])
plt.show()

In [ ]:
plt.plot(history_1.history['mean_absolute_error'])
plt.plot(history_1.history['val_mean_absolute_error'])
plt.title('Model mean_absolute_error')
plt.xlabel('Epoch')
plt.ylabel('mean_absolute_error')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

### Using a Simple RNN

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model_2 = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

In [ ]:
#ERROR_CODE 

model.summary() 

In [ ]:
## ERROR_CODE 

tf.keras.utils.plot_model(model,  show_shapes=True)

In [ ]:
a1 = history_1.history['loss'][np.argmin(history_1.history['loss'])]
print('The loss of model', a1)

In [ ]:
import pandas as pd

In [ ]:
models = pd.DataFrame({
    'Model': ['Baseline'],
    'Score': [a1]})
models.sort_values(by='Score', ascending=True)

## metric for regression 

A metric is a function that is used to judge the performance of your model.

Metric functions are similar to loss functions, except that the results from evaluating a metric are not used when training the model. Note that you may use any loss function as a metric

- MeanSquaredError 
- RootMeanSquaredError 
- MeanAbsoluteError 
- MeanAbsolutePercentageError

 - rmse = tf.keras.metrics.RootMeanSquaredError()
 - mae = tf.keras.metrics.MeanAbsoluteError()


In [ ]:
###################
# ERROR CODE (1)
######################
model_2.compile(loss="mse", optimizer=keras.optimizers.Adam(lr=0.01), metrics=[   (rmse, mae)   ])

In [ ]:
history_2 = model_2.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

In [ ]:
model_2.evaluate(X_valid, y_valid)

In [ ]:
plot_learning_curves(history_2.history["loss"], history_2.history["val_loss"])
plt.show()

In [ ]:
y_pred = model_2.predict(X_valid)
plot_series(X_valid[0, :, 0],windows,y_valid[0, 0], y_pred[0, 0])
plt.show()

### model 3: Deep RNNs with return_sequences=True

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model_3 = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(horizon)
])

model_3.compile(loss="mse", optimizer=keras.optimizers.Adam(lr=0.01), metrics=[last_time_step_mse])
history_3 = model_3.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

In [ ]:
model_3.evaluate(X_valid, y_valid)

In [ ]:
plot_learning_curves(history_3.history["loss"], history_3.history["val_loss"])
plt.show()

In [ ]:
y_pred = model_3.predict(X_valid)
plot_series(X_valid[0, :, 0], windows,y_valid[0, 0], y_pred[0, 0])
plt.show()

### model 4: the second SimpleRNN layer return only the last output

Make the second `SimpleRNN` layer return only the last output:

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model_4 = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(horizon)
])

model_4.compile(loss="mse", optimizer=keras.optimizers.Adam(lr=0.01), metrics=[last_time_step_mse])
history_4 = model_4.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

In [ ]:
model_4.evaluate(X_valid, y_valid)

In [ ]:
plot_learning_curves(history_4.history["loss"], history_4.history["val_loss"])
plt.show()

In [ ]:
y_pred = model_4.predict(X_valid)
plot_series(X_valid[0, :, 0], windows, y_valid[0, 0], y_pred[0, 0])
plt.show()